<a href="https://colab.research.google.com/github/sandrineyb/data-game/blob/main/gamestop_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import requests
import json
import pandas as pd
import time
from datetime import datetime

In [20]:
api_key = '25302ea7a0a560abf4bc4357c4b2b42f2fb19413'
url = f'http://www.gamespot.com/api/games/?api_key={api_key}'

def scrape_gamespot_games(api_key, limit_per_page=100, max_pages=None, delay=1):
    """
    Scrape GameSpot games API and return DataFrame

    Parameters:
    - api_key: Your GameSpot API key
    - limit_per_page: Number of results per page (max 100)
    - max_pages: Maximum number of pages to scrape (None for all)
    - delay: Delay between requests in seconds
    """

    all_games = []
    offset = 0
    page = 1

    print("Starting GameSpot API scraping...")

    while True:
        params = {
            'api_key': api_key,
            'format': 'json',
            'limit': limit_per_page,
            'offset': offset,
            'field_list': 'id,name,deck,genres,platforms,original_release_date,image,site_detail_url,expected_release_year,expected_release_month,expected_release_day'
        }

        try:
            print(f"Fetching page {page} (offset: {offset})...")
            response = requests.get(url, params=params, timeout=15)
            response.raise_for_status()

            data = response.json()


            if 'results' not in data or not data['results']:
                print("No more results found.")
                break

            games = data['results']
            all_games.extend(games)

            print(f"Page {page}: Retrieved {len(games)} games")
            print(f"Total games so far: {len(all_games)}")


            if len(games) < limit_per_page:
                print("Reached last page (fewer results than limit)")
                break

            if max_pages and page >= max_pages:
                print(f"Reached maximum pages limit: {max_pages}")
                break


            offset += limit_per_page
            page += 1


            time.sleep(delay)

        except requests.exceptions.RequestException as e:
            print(f"Error fetching page {page}: {e}")
            break
        except KeyError as e:
            print(f"Unexpected response structure: {e}")
            print(f"Response keys: {data.keys() if 'data' in locals() else 'No data'}")
            break
        except Exception as e:
            print(f"Unexpected error: {e}")
            break

    print(f"\nScraping completed! Total games retrieved: {len(all_games)}")


    if all_games:
        df = pd.DataFrame(all_games)
        return clean_gamespot_dataframe(df)
    else:
        return pd.DataFrame()

def clean_gamespot_dataframe(df):
    print("Cleaning and processing data...")


    if 'platforms' in df.columns:

        df['platform_names'] = df['platforms'].apply(
            lambda x: [platform.get('name', '') for platform in x] if isinstance(x, list) else []
        )
        df['platform_count'] = df['platform_names'].apply(len)

    if 'genres' in df.columns:

        df['genre_names'] = df['genres'].apply(
            lambda x: [genre.get('name', '') for genre in x] if isinstance(x, list) else []
        )
        df['genre_count'] = df['genre_names'].apply(len)


    if 'image' in df.columns:
        df['image_url'] = df['image'].apply(
            lambda x: x.get('super_url', '') if isinstance(x, dict) else ''
        )


    date_columns = ['original_release_date']
    for col in date_columns:
        if col in df.columns:
            try:
                df[col] = pd.to_datetime(df[col], errors='coerce')
            except:
                pass


    if 'original_release_date' in df.columns:
        df['release_year'] = df['original_release_date'].dt.year

    if 'original_release_date' in df.columns:
        df = df.sort_values('original_release_date', ascending=False, na_last=True)

    print(f"Data cleaning completed!")
    print(f"DataFrame shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")

    return df

def display_sample_data(df, n=5):
    """
    Display sample data and basic statistics
    """
    if df.empty:
        print("No data to display")
        return

    print(f"\n--- SAMPLE DATA (First {n} rows) ---")
    cols_to_display = ['name']
    if 'release_year' in df.columns:
        cols_to_display.append('release_year')
    if 'platform_count' in df.columns:
        cols_to_display.append('platform_count')
    if 'genre_count' in df.columns:
        cols_to_display.append('genre_count')

    print(df[cols_to_display].head(n))

    print(f"\n--- BASIC STATISTICS ---")
    print(f"Total games: {len(df)}")

    if 'release_year' in df.columns:
        print(f"Release years range: {df['release_year'].min():.0f} - {df['release_year'].max():.0f}")

    if 'platform_count' in df.columns:
        print(f"Average platforms per game: {df['platform_count'].mean():.1f}")

    if 'genre_count' in df.columns:
        print(f"Average genres per game: {df['genre_count'].mean():.1f}")


if __name__ == "__main__":
    games_df = scrape_gamespot_games(
        api_key=api_key,
        limit_per_page=100,
        max_pages=5,
        delay=1
    )


    if not games_df.empty:
        display_sample_data(games_df)


        filename = f"gamespot_games.csv"
        games_df.to_csv(filename, index=False)
        print(f"\nData saved to: {filename}")

        print(f"\n--- DATA TYPES ---")
        print(games_df.dtypes)
    else:
        print("No data was retrieved. Please check your API key and connection.")

Starting GameSpot API scraping...
Fetching page 1 (offset: 0)...
Page 1: Retrieved 100 games
Total games so far: 100
Fetching page 2 (offset: 100)...
Page 2: Retrieved 100 games
Total games so far: 200
Fetching page 3 (offset: 200)...
Error fetching page 3: HTTPSConnectionPool(host='www.gamespot.com', port=443): Read timed out. (read timeout=15)

Scraping completed! Total games retrieved: 200
Cleaning and processing data...
Data cleaning completed!
DataFrame shape: (200, 9)
Columns: ['id', 'name', 'deck', 'image', 'genres', 'site_detail_url', 'genre_names', 'genre_count', 'image_url']

--- SAMPLE DATA (First 5 rows) ---
                                       name  genre_count
0                         Command & Conquer            2
1                          Return of Arcade            1
2                                   The Dig            1
3                          Mission Critical            1
4  Links LS: Legends in Sports 1997 Edition            3

--- BASIC STATISTICS ---
Tota

In [21]:
games_df.head()

,id,name,deck,image,genres,site_detail_url,genre_names,genre_count,image_url
0,1,Command & Conquer,Overall Command & Conquer is a great strategy ...,{'square_tiny': 'https://www.gamespot.com/a/up...,"[{'name': 'Real-Time'}, {'name': 'Strategy'}]",https://www.gamespot.com/command-and-conquer/,"[Real-Time, Strategy]",2,
1,2,Return of Arcade,"For nostalgia buffs, these old games are worth...",{'square_tiny': 'https://www.gamespot.com/a/up...,[{'name': 'Compilation'}],https://www.gamespot.com/return-of-arcade/,[Compilation],1,
2,3,The Dig,"In almost every sense, The Dig represents a le...",{'square_tiny': 'https://www.gamespot.com/a/up...,[{'name': 'Adventure'}],https://www.gamespot.com/the-dig/,[Adventure],1,
3,4,Mission Critical,"Put simply, Mission Critical was one of the be...",{'square_tiny': 'https://www.gamespot.com/a/up...,[{'name': 'Adventure'}],https://www.gamespot.com/mission-critical/,[Adventure],1,
4,5,Links LS: Legends in Sports 1997 Edition,"With this update, Links LS can rightfully recl...",{'original': 'https://www.gamespot.com/a/uploa...,"[{'name': 'Golf'}, {'name': 'Simulation'}, {'n...",https://www.gamespot.com/links-ls-legends-in-s...,"[Golf, Simulation, Sports]",3,
